In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.3.0


In [4]:
model = load_model('best_model.hdf5')
model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 25, 64)       12800       input_9[0][0]                    
__________________________________________________________________________________________________
dropout_12 (Dropout)            (None, 25, 64)       0           embedding_4[0][0]                
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 86)]         0                                            
_______________________________________________________________________________________

In [11]:
data = pd.read_excel('Label_encoded.xlsx')
df = data.copy()
df = df.drop('Unnamed: 0', axis=1)
df.head()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff,le_texts
0,Konex,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141,"146, 59, 87, 66, 15, 72, 115, 112, 39, 63, 36,..."
1,Finjob,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45,2016-01-04,2016-11-02,303,"85, 153, 12, 62, 165, 113, 33"
2,Kinnamplus,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1054,2016-01-06,2016-12-07,336,"35, 2, 148, 27, 43, 0, 96, 100, 132, 116, 138,..."
3,Genco Pura Olive Oil Company,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,"52, 37, 150, 118, 99, 133, 42"
4,Initech,wilburn@piedpiper.com,alejandro@initech.com,Lost,MG Advanced,517,2016-01-06,2016-09-18,256,"102, 119, 171, 157, 148"


In [12]:
df.insert(10, 'count', 0)
for i in range(df.shape[0]):
    texts = str(df.iloc[i]['le_texts']).split(', ')
    df.iat[i, 10] = len(texts)
df.head()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff,le_texts,count
0,Konex,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141,"146, 59, 87, 66, 15, 72, 115, 112, 39, 63, 36,...",28
1,Finjob,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45,2016-01-04,2016-11-02,303,"85, 153, 12, 62, 165, 113, 33",7
2,Kinnamplus,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1054,2016-01-06,2016-12-07,336,"35, 2, 148, 27, 43, 0, 96, 100, 132, 116, 138,...",15
3,Genco Pura Olive Oil Company,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,"52, 37, 150, 118, 99, 133, 42",7
4,Initech,wilburn@piedpiper.com,alejandro@initech.com,Lost,MG Advanced,517,2016-01-06,2016-09-18,256,"102, 119, 171, 157, 148",5


In [13]:
df.insert(4, 'Stage_BN', 0)
for i in range(df.shape[0]):
    stage = str(df.iloc[i]['Deal_Stage'])
    if stage == 'Won':
        df.iat[i, 4] = 1
    else:
        df.iat[i, 4] = 0
df.head()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Stage_BN,Product,Close_Value,Created Date,Close Date,DateDiff,le_texts,count
0,Konex,corliss@piedpiper.com,delila@konex.com,Won,1,GTX Plus Basic,1255,2016-01-04,2016-05-24,141,"146, 59, 87, 66, 15, 72, 115, 112, 39, 63, 36,...",28
1,Finjob,rosalina@piedpiper.com,belinda@finjob.com,Won,1,MG Special,45,2016-01-04,2016-11-02,303,"85, 153, 12, 62, 165, 113, 33",7
2,Kinnamplus,donn@piedpiper.com,monte@kinnamplus.com,Lost,0,MG Special,1054,2016-01-06,2016-12-07,336,"35, 2, 148, 27, 43, 0, 96, 100, 132, 116, 138,...",15
3,Genco Pura Olive Oil Company,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,0,MG Advanced,1054,2016-01-06,2016-09-11,249,"52, 37, 150, 118, 99, 133, 42",7
4,Initech,wilburn@piedpiper.com,alejandro@initech.com,Lost,0,MG Advanced,517,2016-01-06,2016-09-18,256,"102, 119, 171, 157, 148",5


In [28]:
def get_padded(text):
    extra = np.array([0]*25)
    try:
        text = list(map(int, str(text).split(', ')))
    except:
        text = list()
    arr = np.concatenate((text, extra))[:25]
    return arr

In [16]:
df = pd.concat([df, pd.get_dummies(df['Account'], prefix='account=')], axis=1)
df.head()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Stage_BN,Product,Close_Value,Created Date,Close Date,DateDiff,...,account=_Xx-holding,account=_Xx-zobam,account=_Y-corporation,account=_Yearin,account=_Zathunicon,account=_Zencorporation,account=_Zoomit,account=_Zotware,account=_Zumgoity,account=_dambase
0,Konex,corliss@piedpiper.com,delila@konex.com,Won,1,GTX Plus Basic,1255,2016-01-04,2016-05-24,141,...,0,0,0,0,0,0,0,0,0,0
1,Finjob,rosalina@piedpiper.com,belinda@finjob.com,Won,1,MG Special,45,2016-01-04,2016-11-02,303,...,0,0,0,0,0,0,0,0,0,0
2,Kinnamplus,donn@piedpiper.com,monte@kinnamplus.com,Lost,0,MG Special,1054,2016-01-06,2016-12-07,336,...,0,0,0,0,0,0,0,0,0,0
3,Genco Pura Olive Oil Company,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,0,MG Advanced,1054,2016-01-06,2016-09-11,249,...,0,0,0,0,0,0,0,0,0,0
4,Initech,wilburn@piedpiper.com,alejandro@initech.com,Lost,0,MG Advanced,517,2016-01-06,2016-09-18,256,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_inputs = df[df['Deal_Stage']=='In Progress']
df_inputs.head()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Stage_BN,Product,Close_Value,Created Date,Close Date,DateDiff,...,account=_Xx-holding,account=_Xx-zobam,account=_Y-corporation,account=_Yearin,account=_Zathunicon,account=_Zencorporation,account=_Zoomit,account=_Zotware,account=_Zumgoity,account=_dambase
3,Genco Pura Olive Oil Company,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,0,MG Advanced,1054,2016-01-06,2016-09-11,249,...,0,0,0,0,0,0,0,0,0,0
15,Umbrella Corporation,daniell@piedpiper.com,orpha@umbrellacorporation.com,In Progress,0,GTX Plus Basic,561,2016-01-17,2016-10-03,260,...,0,0,0,0,0,0,0,0,0,0
35,Ontomedia,markita@piedpiper.com,nedra@ontomedia.com,In Progress,0,MG Special,50,2016-02-13,2016-11-01,262,...,0,0,0,0,0,0,0,0,0,0
39,Hottechi,kary@piedpiper.com,juliette@hottechi.com,In Progress,0,GTX Plus Basic,601,2016-02-20,2016-05-30,100,...,0,0,0,0,0,0,0,0,0,0
42,Silis,cassey@piedpiper.com,zulema@silis.com,In Progress,0,MG Advanced,560,2016-02-22,2016-09-16,207,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_inputs.shape

(2089, 97)

In [27]:
df_inputs['le_texts'].isnull().value_counts()

False    1996
True       93
Name: le_texts, dtype: int64

In [23]:
def data_generator_no_op(df,i):
    row = df.iloc[i]
    text = np.array(get_padded(row['le_texts'])).astype(float)
    extra = np.array(np.append(row['Close_Value'], row[list(df.columns[12:])])).astype(float)
    #output = row['Stage_BN'].astype(float)
    return text, extra

In [29]:
inputs_text = list()
inputs_extra = list()
for i in range(df_inputs.shape[0]):
    text, extra = data_generator_no_op(df_inputs, i)
    inputs_text.append(text)
    inputs_extra.append(extra)

In [33]:
inputs_text = np.array(inputs_text)
inputs_extra = np.array(inputs_extra)

In [34]:
print(inputs_text.shape)
print(inputs_extra.shape)

(2089, 25)
(2089, 86)


In [45]:
def generate_input(df, i):
    text, extra = data_generator_no_op(df, i)
    inputs_text = np.array(text)
    inputs_extra = np.array(extra)
    return inputs_text, inputs_extra

In [46]:
df.head()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Stage_BN,Product,Close_Value,Created Date,Close Date,DateDiff,...,account=_Xx-holding,account=_Xx-zobam,account=_Y-corporation,account=_Yearin,account=_Zathunicon,account=_Zencorporation,account=_Zoomit,account=_Zotware,account=_Zumgoity,account=_dambase
0,Konex,corliss@piedpiper.com,delila@konex.com,Won,1,GTX Plus Basic,1255,2016-01-04,2016-05-24,141,...,0,0,0,0,0,0,0,0,0,0
1,Finjob,rosalina@piedpiper.com,belinda@finjob.com,Won,1,MG Special,45,2016-01-04,2016-11-02,303,...,0,0,0,0,0,0,0,0,0,0
2,Kinnamplus,donn@piedpiper.com,monte@kinnamplus.com,Lost,0,MG Special,1054,2016-01-06,2016-12-07,336,...,0,0,0,0,0,0,0,0,0,0
3,Genco Pura Olive Oil Company,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,0,MG Advanced,1054,2016-01-06,2016-09-11,249,...,0,0,0,0,0,0,0,0,0,0
4,Initech,wilburn@piedpiper.com,alejandro@initech.com,Lost,0,MG Advanced,517,2016-01-06,2016-09-18,256,...,0,0,0,0,0,0,0,0,0,0


In [48]:
for i in range(df.shape[0]):
    row = df.iloc[i]
    if row['Deal_Stage']=='In Progress':
        text,extra = generate_input(df, i)
        text = np.array([text])
        extra = np.array([extra])
        pred = model.predict(x = [text,extra])
        pred = int(pred>0.6)
        df.iat[i, 4] = pred
        if pred==1:
            df.iat[i, 3] = 'Won'
        else:
            df.iat[i, 3] = 'Lost'
df.head()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Stage_BN,Product,Close_Value,Created Date,Close Date,DateDiff,...,account=_Xx-holding,account=_Xx-zobam,account=_Y-corporation,account=_Yearin,account=_Zathunicon,account=_Zencorporation,account=_Zoomit,account=_Zotware,account=_Zumgoity,account=_dambase
0,Konex,corliss@piedpiper.com,delila@konex.com,Won,1,GTX Plus Basic,1255,2016-01-04,2016-05-24,141,...,0,0,0,0,0,0,0,0,0,0
1,Finjob,rosalina@piedpiper.com,belinda@finjob.com,Won,1,MG Special,45,2016-01-04,2016-11-02,303,...,0,0,0,0,0,0,0,0,0,0
2,Kinnamplus,donn@piedpiper.com,monte@kinnamplus.com,Lost,0,MG Special,1054,2016-01-06,2016-12-07,336,...,0,0,0,0,0,0,0,0,0,0
3,Genco Pura Olive Oil Company,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,Won,1,MG Advanced,1054,2016-01-06,2016-09-11,249,...,0,0,0,0,0,0,0,0,0,0
4,Initech,wilburn@piedpiper.com,alejandro@initech.com,Lost,0,MG Advanced,517,2016-01-06,2016-09-18,256,...,0,0,0,0,0,0,0,0,0,0


In [49]:
df.tail()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Stage_BN,Product,Close_Value,Created Date,Close Date,DateDiff,...,account=_Xx-holding,account=_Xx-zobam,account=_Y-corporation,account=_Yearin,account=_Zathunicon,account=_Zencorporation,account=_Zoomit,account=_Zotware,account=_Zumgoity,account=_dambase
8795,Vehement Capital Partners,jonathan@piedpiper.com,lavonia@vehementcapitalpartners.com,Won,1,GTX Basic,590,2018-12-30,2019-07-22,204,...,0,0,0,0,0,0,0,0,0,0
8796,Hottechi,marty@piedpiper.com,juliette@hottechi.com,Won,1,MG Advanced,3284,2018-12-30,2019-05-26,147,...,0,0,0,0,0,0,0,0,0,0
8797,Umbrella Corporation,anna@piedpiper.com,orpha@umbrellacorporation.com,Lost,0,GTX Plus Pro,300,2018-12-31,2019-04-15,105,...,0,0,0,0,0,0,0,0,0,0
8798,Rundofase,gladys@piedpiper.com,crista@rundofase.com,Won,1,MG Special,3284,2018-12-31,2019-05-16,136,...,0,0,0,0,0,0,0,0,0,0
8799,Y-corporation,jonathan@piedpiper.com,rosanne@y-corporation.com,Won,1,MG Special,1014,2018-12-31,2019-08-01,213,...,0,0,1,0,0,0,0,0,0,0


In [61]:
df.to_excel('Predicted_Sales.xlsx')

NameError: name 'read_excel' is not defined

In [71]:
df = pd.read_excel('Predicted_Sales.xlsx')

In [72]:
df.head()

,Unnamed: 0,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Stage_BN,Product,Close_Value,Created Date,Close Date,...,account=_Xx-holding,account=_Xx-zobam,account=_Y-corporation,account=_Yearin,account=_Zathunicon,account=_Zencorporation,account=_Zoomit,account=_Zotware,account=_Zumgoity,account=_dambase
0,8,Gogozoom,darcel@piedpiper.com,salina@gogozoom.com,Won,1,GTXPro,4359,2016-01-10,2016-01-28,...,0,0,0,0,0,0,0,0,0,0
1,5,Stanredtax,lajuana@piedpiper.com,candice@stanredtax.com,Won,1,MG Advanced,3180,2016-01-07,2016-01-31,...,0,0,0,0,0,0,0,0,0,0
2,12,Scotfind,elease@piedpiper.com,gordon@scotfind.com,Won,1,GTX Plus Basic,1164,2016-01-14,2016-02-01,...,0,0,0,0,0,0,0,0,0,0
3,17,Singletechno,maureen@piedpiper.com,tambra@singletechno.com,Won,1,GTX Plus Basic,1070,2016-01-19,2016-02-11,...,0,0,0,0,0,0,0,0,0,0
4,20,Fasehatice,kary@piedpiper.com,hsiu@fasehatice.com,Won,1,GTXPro,5149,2016-01-22,2016-03-16,...,0,0,0,0,0,0,0,0,0,0


In [78]:
def get_revenue(df, start, end):
    df = df.sort_values(by=['Close Date'])
    won = df[(df['Close Date'] >= start) & (df['Close Date'] <= end) & (df['Deal_Stage'] == 'Won')]
    total_revenue = sum(won['Close_Value'])
    return total_revenue

In [80]:
# For current month
start = pd.Timestamp('2019-06-01')
end = pd.Timestamp('2019-06-30')
print('Total revenue for the month of June 2019: ', get_revenue(df, start, end))

# For current quarter
start = pd.Timestamp('2019-04-01')
end = pd.Timestamp('2019-06-30')
print('Total revenue for the 2nd quarter of 2019: ', get_revenue(df, start, end))

# For current year
start = pd.Timestamp('2019-01-01')
end = pd.Timestamp('2019-12-31')
print('Total revenue for 2019: ', get_revenue(df, start, end))

Total revenue for the month of June 2019:  318231
Total revenue for the 2nd quarter of 2019:  858665
Total revenue for 2019:  3711364
